In [2]:
install.packages("bda")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [3]:
library(conflicted)
library(readxl)
library(gtools)
library(plm)
library(tidyverse)
library(modelsummary)
library(huxtable)
library(dplyr)
library(bda)
library(writexl)
library(Hmisc)
library(sjPlot)
library(DescTools)
library(psych)

conflict_prefer("lag", "stats")
conflict_prefer("Mean", "modelsummary")
conflict_prefer("N", "modelsummary")
conflict_prefer("SD", "modelsummary")

Loading required package: boot


Attaching package: ‘boot’


The following objects are masked from ‘package:gtools’:

    inv.logit, logit


bda v15 (Bin Wang, 2021)

[conflicted] Will prefer stats::lag over any other package.
[conflicted] Will prefer modelsummary::Mean over any other package.
[conflicted] Will prefer modelsummary::N over any other package.
[conflicted] Will prefer modelsummary::SD over any other package.


In [12]:
output_file_path <- file.path(path.expand('~'), 'OneDrive/PhD Dissertation/Data_Code/Data/summary.txt')

df <- pdata.frame(read.csv(file.path(path.expand('~'), 'OneDrive/PhD Dissertation/Data_Code/Data/index_data_regvariables.csv')),
                  index=c("Numeric","Year"))

df["lnFDIstock_w"] <- as.vector(winsor(df["lnFDIstock"], trim = .01))

In [13]:
y <- "SDI~"
x_s <- c(
    "lag(SDI)",
    "lnFDIstock_w",
    "WGI",
    "urban",
    "tax",
    "war_intensity",
    "financial"
)

gmm_iv <- "lag(SDI)"
ivs <- c(
    "lnFDIstock_w",
    "WGI",
    "urban",
    "tax",
    "war_intensity",
    "financial"
)

In [15]:
formulas <- formula()

for (l in seq_len(length(ivs))) {
    comb <- combinations(n = length(ivs), r = l, v = ivs)
    year_dummies = c()

    for (i in seq_len(dim(comb)[1])) {
        iv <- paste(comb[i, ], collapse = "+")
        x <- paste(
            c(paste(x_s, collapse = "+"), gmm_iv, iv),
            collapse = "|"
        )
        f <- as.formula(paste(y, x))

        formulas <- append(formulas, f)
    }
}

results <- lapply(formulas, function(f) {
    tryCatch(plm::pgmm(
        f,
        data = df,
        effect = "individual",
        model = "twosteps",
        collapse = FALSE,
        transformation = "ld",
    ), error = function(e) NULL)
})

count <- 0
sink(output_file_path)
sink()

is.acceptable <- function(asummary) {
  sargan_p_value <- asummary$sargan$p.value[["chisq"]]
  ar1 <- asummary$m1$p.value[1]
  ar2 <- asummary$m2$p.value[1]
  scount <- sum(asummary$coefficients[, 4] < 0.10)
  core_p_value <- asummary$coefficients[1, 4]
  core_p_value_2 <- asummary$coefficients[2, 4]
  core_p_value_3 <- asummary$coefficients[3, 4]
  # core_p_value_4 <- asummary$coefficients[4, 4]
  core_coef_2 <- asummary$coefficients[2, 1]
  core_coef_3 <- asummary$coefficients[3, 1]
  # core_coef_4 <- asummary$coefficients[4, 1]
  return(
    sargan_p_value >= 0.15 &
    sargan_p_value < 0.5 &
    !is.nan(ar1) & ar1 < 0.1 &
    !is.nan(ar2) & ar2 >= 0.05 &
    scount >= 1 &
    core_p_value_2 < 0.05 &
    core_coef_2 > 0 &
    core_coef_3 > 0 &
    core_p_value < 0.01 &
    !is.na(scount) & !is.nan(core_p_value) &
    TRUE
  )
}

for (i in seq_len(length(results))) {
    result <- results[[i]]
    print(i)
    if (!result %>% is.null()) {
        print(result)
        asummary <- result %>% summary(robust = TRUE, time.dummies = TRUE)
        if (asummary %>% is.acceptable()) {
            count <- count + 1
            sink(output_file_path, append = TRUE)
            print(formulas[[i]])
            print(asummary)
            cat("________________________________________\n")
            sink()
        }
    }
}

cat(c("Count:", count, "\n"), file = output_file_path, append = TRUE)

Warning message in plm::pgmm(f, data = df, effect = "individual", model = "twosteps", :
“the second-step matrix is singular, a general inverse is used”
Warning message in plm::pgmm(f, data = df, effect = "individual", model = "twosteps", :
“the second-step matrix is singular, a general inverse is used”
Warning message in plm::pgmm(f, data = df, effect = "individual", model = "twosteps", :
“the second-step matrix is singular, a general inverse is used”
Warning message in plm::pgmm(f, data = df, effect = "individual", model = "twosteps", :
“the second-step matrix is singular, a general inverse is used”
Warning message in plm::pgmm(f, data = df, effect = "individual", model = "twosteps", :
“the second-step matrix is singular, a general inverse is used”
Warning message in plm::pgmm(f, data = df, effect = "individual", model = "twosteps", :
“the second-step matrix is singular, a general inverse is used”
Warning message in plm::pgmm(f, data = df, effect = "individual", model = "twosteps", :


[1] 1

Model Formula: SDI ~ lag(SDI) + lnFDIstock_w + WGI + urban + tax + war_intensity + 
    financial | lag(SDI) | WGI + financial + lnFDIstock_w + tax + 
    urban + war_intensity

Coefficients:
     lag(SDI)  lnFDIstock_w           WGI         urban           tax 
    0.8907589     0.0032269    -0.0049091     0.0032895     0.2874497 
war_intensity     financial 
   -0.0047873    -0.0010780 



Warning message in vcovHC.pgmm(object):
“a general inverse is used”


[1] 2

Model Formula: SDI ~ lag(SDI) + lnFDIstock_w + WGI + urban + tax + war_intensity + 
    financial | lag(SDI) | WGI + financial + lnFDIstock_w + tax + 
    urban + war_intensity

Coefficients:
     lag(SDI)  lnFDIstock_w           WGI         urban           tax 
   0.91541942    0.00089552   -0.00304183    0.00494772    0.26281103 
war_intensity     financial 
  -0.00279628    0.00033954 



Warning message in vcovHC.pgmm(object):
“a general inverse is used”


...

Warning message in vcovHC.pgmm(object):
“a general inverse is used”


[1] 62

Model Formula: SDI ~ lag(SDI) + lnFDIstock_w + WGI + urban + tax + war_intensity + 
    financial | lag(SDI) | WGI + financial + lnFDIstock_w + tax + 
    urban + war_intensity

Coefficients:
     lag(SDI)  lnFDIstock_w           WGI         urban           tax 
   9.9358e-01    2.2653e-04   -1.9692e-03    6.8922e-05    8.7448e-03 
war_intensity     financial 
  -9.4879e-04    2.9997e-04 



Warning message in vcovHC.pgmm(object):
“a general inverse is used”


[1] 63

Model Formula: SDI ~ lag(SDI) + lnFDIstock_w + WGI + urban + tax + war_intensity + 
    financial | lag(SDI) | WGI + financial + lnFDIstock_w + tax + 
    urban + war_intensity

Coefficients:
     lag(SDI)  lnFDIstock_w           WGI         urban           tax 
   9.9275e-01    3.8641e-04   -8.9517e-04    7.0268e-05    8.5388e-03 
war_intensity     financial 
  -7.7142e-04    2.0826e-04 



Warning message in vcovHC.pgmm(object):
“a general inverse is used”


In [9]:
plm::pgmm(
        f,
        data = df,
        effect = "individual",
        model = "twosteps",
        collapse = FALSE,
        transformation = "ld",
    )

ERROR: Error in eval(predvars, data, env): object 'wardummy' not found
